<a href="https://colab.research.google.com/github/pradiptha/files/blob/master/Indonesian_Twitter_Sentiment_Analysis_with_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Analisis Sentimen terhadap data tweet berbahasa Indonesia dengan label (-1, 0, 1)**

In [ ]:
# print('downloading data...')
# !wget https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv
# !wget https://raw.githubusercontent.com/pradiptha/files/master/Indonesian%20Sentiment%20Twitter%20Dataset%20Labeled.csv
# !wget https://raw.githubusercontent.com/pradiptha/files/master/Preproccessed-data.csv

In [ ]:
# !pip install sastrawi
# import nltk
# nltk.download('stopwords')

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from multiprocessing import Pool
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re
import string
import time
import psutil
import pandas as pd
import numpy as np
from os import path
seed = 7
np.random.seed(seed)
listStopword = set(stopwords.words('indonesian'))
factory = StemmerFactory()
stemmerIndonesia = factory.create_stemmer()

In [ ]:
def readDataset(filename):
    df = pd.read_csv(filename, sep="\t")
    target = list(df["sentimen"])
    data = list(df["Tweet"])
    target = target
    data = data
    return data, target

def preProcessing(tweet):
    tweet = tweet.lower()  # Convert text to lowercase
    tweet = re.sub(r'\d+', '', tweet)  # Numbers removing
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)  # funnnnny --> funny
    tweet = re.sub(r'(.)\1{2,}', r'\1', tweet)
    tweet = tweet.translate(str.maketrans("", "", string.punctuation))
    tweet = tweet.strip(r'\n')
    tweet = tweet.strip(' ')  # White spaces removal
    tweet = normalization(tweet)
    # tweet = nltk.tokenize.word_tokenize(tweet)
    filtering = [i for i in list(tweet) if not i in listStopword]
    tweet = [term for term in filtering if (term != "" and term != "-")]
    tweet = " ".join(tweet)
    tweet = stemmerIndonesia.stem(tweet)
    return tweet

def savePreProcessing(sentimen, tweet):
    sentimen_a = []
    tweet_a = []
    for i in range(len(tweet)):
        if tweet[i] is not "":
            tweet_a.append(tweet[i])
            sentimen_a.append(sentimen[i])

    data = {'sentimen': sentimen_a, 'Tweet': tweet_a}
    print("saving preprocessed data ...")
    pd.DataFrame(data).to_csv("Preproccessed-data.csv", sep="\t", index=False)


def normalization(tweet):
    df = pd.read_csv("colloquial-indonesian-lexicon.csv", sep=",")
    slang = list(df["slang"])
    formal = list(df["formal"])
    tweet = tweet.split()
    new_tweet = []
    for i in tweet:
        for index, j in enumerate(slang):
            if i == j:
                i = formal[index]
        new_tweet.append(i)
    # new_tweet = ' '.join(new_tweet)
    return(new_tweet)

def fiturExtract(x_train, x_test):
    tfidf_vectorizer = TfidfVectorizer()
    x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
    x_test_tfidf = tfidf_vectorizer.transform(x_test)
    return x_train_tfidf, x_test_tfidf

In [ ]:
print("Sentiment Analysis Using Machine Learning")
print("-----------------------------------------\n")
print("Reading the DataSet...")
if path.exists('Preproccessed-data.csv'):
    tweets, target = readDataset('Preproccessed-data.csv')
else:
    data, target = readDataset('Indonesian Sentiment Twitter Dataset Labeled.csv')
    print("-----------------------------------------\n")
    print("Preprocessing the DataSet. This may take some time...")
    num_cpus = psutil.cpu_count()
    p = Pool(num_cpus)
    tweets = p.map(preProcessing, data)
    p.close()
    savePreProcessing(target, tweets)
print("done")

Sentiment Analysis Using Machine Learning
-----------------------------------------

Reading the DataSet...
done


In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(target)
encoded_Y = encoder.transform(target)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)
print('sebelum\n', target[:5])
print('sesudah\n', dummy_y[:5])

sebelum
 [-1, -1, 1, 1, -1]
sesudah
 [[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    tweets, dummy_y, test_size=0.3, random_state=10)
print(f"Data train {len(x_train)} entries, Data test {len(x_test)} entries")

Data train 7491 entries, Data test 3211 entries


In [ ]:
x_train_tfidf, x_test_tfidf = fiturExtract(x_train, x_test)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=x_train_tfidf.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 128)               1661952   
_________________________________________________________________
dense_38 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_39 (Dense)             (None, 3)                 195       
Total params: 1,670,403
Trainable params: 1,670,403
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train_tfidf, y_train, batch_size=128, epochs=10, validation_data=(x_test_tfidf, y_test))

Train on 7491 samples, validate on 3211 samples
Epoch 1/10
7491/7491 [==============================] - 3s 368us/step - loss: 1.0345 - accuracy: 0.4905 - val_loss: 0.9776 - val_accuracy: 0.4858
Epoch 2/10
7491/7491 [==============================] - 3s 346us/step - loss: 0.7632 - accuracy: 0.6528 - val_loss: 0.8885 - val_accuracy: 0.6145
Epoch 3/10
7491/7491 [==============================] - 3s 355us/step - loss: 0.3730 - accuracy: 0.8690 - val_loss: 1.0728 - val_accuracy: 0.6104
Epoch 4/10
7491/7491 [==============================] - 3s 347us/step - loss: 0.2015 - accuracy: 0.9325 - val_loss: 1.2659 - val_accuracy: 0.5939
Epoch 5/10
7491/7491 [==============================] - 3s 353us/step - loss: 0.1362 - accuracy: 0.9545 - val_loss: 1.4111 - val_accuracy: 0.5830
Epoch 6/10
7491/7491 [==============================] - 3s 353us/step - loss: 0.1045 - accuracy: 0.9625 - val_loss: 1.5385 - val_accuracy: 0.5780
Epoch 7/10
7491/7491 [==============================] - 3s 350us/step - loss

In [ ]:
y_pred = model.predict_classes(x_test_tfidf)
encoder.fit(y_pred)
encoded_Y = encoder.transform(y_pred)
# convert integers to dummy variables (i.e. one hot encoded)
y_pred = to_categorical(encoded_Y)

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_tfidf, y_test)
print("test loss, test acc:", results)

Evaluate on test data
3211/3211 [==============================] - 0s 129us/step
test loss, test acc: [1.8743639431872943, 0.570227324962616]


In [ ]:
y_test_s = np.argmax(y_test, axis=1)
y_pred_s = np.argmax(y_pred, axis=1)
y_test_s = [i - 1 for i in y_test_s]
y_pred_s = [i - 1 for i in y_pred_s]
print("Accuracy_score : ", accuracy_score(y_test_s, y_pred_s))
print("Confusion_matrix : \n", confusion_matrix(y_test_s, y_pred_s))
print(classification_report(y_test_s, y_pred_s))

Accuracy_score :  0.5702273435066957
Confusion_matrix : 
 [[500 283 120]
 [332 994 233]
 [156 256 337]]
              precision    recall  f1-score   support

          -1       0.51      0.55      0.53       903
           0       0.65      0.64      0.64      1559
           1       0.49      0.45      0.47       749

    accuracy                           0.57      3211
   macro avg       0.55      0.55      0.55      3211
weighted avg       0.57      0.57      0.57      3211



In [ ]:
x_train_tfidf.shape[1]

12983

In [ ]:
df = pd.read_csv('Indonesian Sentiment Twitter Dataset Labeled.csv', sep='\t')
sentimen = list(df['sentimen'])
tweet = list(df['Tweet'])
for i in range(len(sentimen)):
  if sentimen[i] == -1:
    print(tweet)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

